# COMPSCI 371D Homework 8

Write the names of all contributors to this assignment here:

+ ...
+ ...

## Part 1: 

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pickle

with open('trivial.pickle', 'rb') as file:
    trivial = pickle.load(file)

### Problem 1.1

In [2]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

def evaluate(h, train, test, description):
    def accuracy(predictor, s):
        sx, sy = s['x'], s['y']
        return predictor.score(sx, sy) * 100

    performance = {
        'description': description,
        'training accuracy': accuracy(h, train),
        'test accuracy': accuracy(h, test)
    }
    f = '{:s}: training accuracy is {:.2f} percent,' +\
        '\ntest accuracy is {:.2f} percent'
    print(f.format(description, performance['training accuracy'],
                   performance['test accuracy']))
    return performance

In [3]:
def experiment(data_set, classifier_name, classifier, hyper_parameters, folds=15, display=False):
    train, test = data_set['train'], data_set['test']
    c = GridSearchCV(classifier, hyper_parameters, scoring='accuracy', cv=folds, iid=False)
    h = c.fit(train['x'], train['y'])
    parm_string = ', '.join(['{} = {}'.format(key, h.best_params_[key])
                             for key in hyper_parameters.keys()])
    description = '{} with {}'.format(classifier_name, parm_string)
    if display:
        show_data(h.best_estimator_, train, description)
    evaluate(h.best_estimator_, train, test, description)

n_folds, display = 15, False
experiment(trivial, 'linear SVM', SVC(kernel='linear', gamma='auto'),
               hyper_parameters={'C': [1000.]}, folds=n_folds, display=display)

linear SVM with C = 1000.0: training accuracy is 100.00 percent,
test accuracy is 100.00 percent


### Problem 1.2 (Exam-Style)

## Part 2: 

In [4]:
import matplotlib.pyplot as plt
%matplotlib inline

with open('data.pickle', 'rb') as file:
    data = pickle.load(file)

### Problem 2.1

In [5]:
c_values = [0.0001, 0.001, 0.01, 0.1, 1., 10., 100.]

In [6]:
n_folds, display = 15, True

### Problem 2.2

### Problem 2.3 (Exam-Style)

### Problem 2.4

### Problem 2.5

### Problem 2.6

### Problem 2.7 (Exam-Style)

### Problem 2.8 (Partially Exam-Style)